In [2]:
%load_ext autoreload
%autoreload 2

In [34]:
import pyrootutils

base_path = pyrootutils.setup_root(
    search_from=".",
    indicator=[".gitignore"],
    project_root_env_var=True,  # set the PROJECT_ROOT environment variable to root directory
    dotenv=True,  # load environment variables from .env if exists in root directory
    pythonpath=True,  # add root directory to the PYTHONPATH (helps with imports)
    cwd=True,  # change current working directory to the root directory (helps with filepaths)
)

import sys
import pandas as pd
import scanpy as sc
import squidpy as sq
import matplotlib.pyplot as plt
import seaborn as sns
import json
import wandb
import matplotlib
import matplotlib as mpl
from itertools import combinations
import os
import shutil
from IPython.display import display
from pprint import pprint

from src.utils.metrics import *
from src.data.utils import *
from src.utils.eval import *
from src.utils.wandb import *
from src.utils.data_paths_valid import *

## Define paths

In [79]:
sc_paths, st_paths, simulation_paths, experiment_paths = get_paths_for_processing(
    base_path, data_dir="data", use_old_experiment_paths=True
)
dataset_names_orig = get_dataset_names()
dataset_map = get_dataset_map(experiment_paths)
dataset_path_map = get_dataset_path_map(st_paths)
dataset_names

['slideSeqv2-kidney-2a',
 'slideSeqv2-kidney-4b',
 'seqFISH-heart-1',
 'seqFISH-heart-2',
 'seqFISH-heart-3',
 'MERFISH-hypothalamus-0',
 'MERFISH-hypothalamus-1',
 'Visium-mouse-brain',
 'Visium-lymph-node',
 'slideSeqv2-kidney-2a (sc. ref.)',
 'slideSeqv2-kidney-4b (sc. ref.)',
 'seqFISH-heart-1 (switched)',
 'seqFISH-heart-2 (switched)',
 'MERFISH-hypothalamus-0 (switched)',
 'MERFISH-hypothalamus-1 (switched)',
 'slideSeqv2-kidney-2a (sc. ref., not var. cut.)',
 'slideSeqv2-kidney-4b (sc. ref., not var. cut.)']

## Visualize data

In [89]:
st_data = sc.read_h5ad(st_paths[-4])
st_data.obs.columns

Index(['row', 'col', 'Ambiguous', 'Astrocyte', 'Endothelial', 'Ependymal',
       'Excitatory', 'Inhibitory', 'Microglia', 'OD Immature', 'OD Mature',
       'Pericytes'],
      dtype='object')

In [91]:
sc_data = sc.read_h5ad(sc_paths[-4])
sc_data.obs["cell_type"].unique()

['Microglia', 'OD Mature', 'Pericytes', 'Ambiguous', 'Inhibitory', 'Astrocyte', 'Endothelial', 'OD Immature', 'Excitatory', 'Ependymal']
Categories (10, object): ['Ambiguous', 'Astrocyte', 'Endothelial', 'Ependymal', ..., 'Microglia', 'OD Immature', 'OD Mature', 'Pericytes']

In [94]:
# merfish_paths = [
#     "./data/spatial/hypothalamus_MERFISH/0.h5ad",
#     "./data/spatial/hypothalamus_MERFISH/1.h5ad",
#     "./data/spatial/hypothalamus_MERFISH/2.h5ad",
# ]
# for path_1 in merfish_paths:
#     data_1 = sc.read_h5ad(path_1)
#     for path_2 in merfish_paths:
#         data_2 = sc.read_h5ad(path_2)
#         print(path_1, path_2, set(data_1.obs["cell_type"].unique()) == set(data_2.obs["cell_type"].unique()))

./data/spatial/hypothalamus_MERFISH/0.h5ad ./data/spatial/hypothalamus_MERFISH/0.h5ad True
./data/spatial/hypothalamus_MERFISH/0.h5ad ./data/spatial/hypothalamus_MERFISH/1.h5ad True
./data/spatial/hypothalamus_MERFISH/0.h5ad ./data/spatial/hypothalamus_MERFISH/2.h5ad True
./data/spatial/hypothalamus_MERFISH/1.h5ad ./data/spatial/hypothalamus_MERFISH/0.h5ad True
./data/spatial/hypothalamus_MERFISH/1.h5ad ./data/spatial/hypothalamus_MERFISH/1.h5ad True
./data/spatial/hypothalamus_MERFISH/1.h5ad ./data/spatial/hypothalamus_MERFISH/2.h5ad True
./data/spatial/hypothalamus_MERFISH/2.h5ad ./data/spatial/hypothalamus_MERFISH/0.h5ad True
./data/spatial/hypothalamus_MERFISH/2.h5ad ./data/spatial/hypothalamus_MERFISH/1.h5ad True
./data/spatial/hypothalamus_MERFISH/2.h5ad ./data/spatial/hypothalamus_MERFISH/2.h5ad True


In [ ]:
# sc_data = sc.read_h5ad(sc_paths[0])
# # convert cell_type collumn into separate dataframe
# for cell_type in sc_data.obs["cell_type"].unique():
#     sc_data.obs[cell_type] = (sc_data.obs["cell_type"] == cell_type).astype(int)
# sc.pl.spatial(sc_data, color=sc_data.obs["cell_type"].unique(), show=True)

## Get results for addtional methods

### Load original DISSECT results

In [41]:
# dissect_results, ensemble_result = load_dissect_results(experiment_path)
ensemble_results = [load_dissect_results(experiment_path)[1] for experiment_path in experiment_paths[0:-2]]
dissect_dataset_names = [extract_dataset_name(path, dataset_map) for path in experiment_paths[0:len(ensemble_results)]]
dissect_names = len(dissect_dataset_names) * ["DISSECT (ensemble)"]

In [82]:
ensemble_results[0]

,blood vessel smooth muscle cell,endothelial cell,kidney collecting duct intercalated cell,kidney collecting duct principal cell,kidney distal convoluted tubule epithelial cell,kidney granular cell,kidney interstitial fibroblast,kidney loop of Henle thick ascending limb epithelial cell,kidney proximal convoluted tubule epithelial cell,leukocyte,macrophage,mesangial cell,podocyte
0_20,0.070197,0.082763,0.076522,0.069793,0.073809,0.065479,0.082292,0.073965,0.093242,0.086581,0.079693,0.074771,0.070891
0_21,0.032877,0.073902,0.013271,0.024880,0.095639,0.012947,0.017085,0.109985,0.543887,0.042311,0.011165,0.009221,0.012830
0_22,0.021406,0.080350,0.023387,0.025803,0.053593,0.026225,0.024170,0.051636,0.575819,0.053099,0.039970,0.011542,0.012999
0_23,0.012979,0.031930,0.021703,0.016811,0.035826,0.024405,0.023215,0.030268,0.736437,0.038487,0.013768,0.006862,0.007310
0_24,0.047739,0.047609,0.039311,0.049376,0.079620,0.013915,0.015661,0.070380,0.467047,0.073546,0.077048,0.008051,0.010697
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9_55,0.075353,0.038479,0.019369,0.043883,0.062504,0.040122,0.053118,0.030117,0.514363,0.073527,0.014630,0.017134,0.017400
9_56,0.039337,0.054429,0.011379,0.043048,0.083926,0.038462,0.023543,0.030904,0.597842,0.030163,0.022021,0.011781,0.013164
9_57,0.079385,0.133796,0.042240,0.055317,0.087703,0.054221,0.084517,0.058029,0.119930,0.081336,0.118101,0.034419,0.051007
9_8,0.098331,0.051842,0.035419,0.071323,0.138031,0.023396,0.030172,0.040219,0.259759,0.138307,0.058554,0.023148,0.031501


### Load RCTD results

In [53]:
rctd_result_paths = [
    "./logs/results/rctd/results/UMOD-WT.WT-2a_resolution75.csv",
    "./logs/results/rctd/results/UMOD-KI.KI-4b_resolution105.csv",
    "./logs/results/rctd/results/embryo1_resolution0.11.csv",
    "./logs/results/rctd/results/embryo2_resolution0.11.csv",
    "./logs/results/rctd/results/embryo3_resolution0.11.csv",
    "./logs/results/rctd/results/st_lymph.csv",
    "./logs/results/rctd/results/UMOD-WT.WT-2a_resolution75_refAtlas.csv",
    "./logs/results/rctd/results/UMOD-KI.KI-4b_resolution105_refAtlas.csv",
    "./logs/results/rctd/results/embryo1_resolution0.11_switch_reference.csv",
    "./logs/results/rctd/results/embryo2_resolution0.11_switch_reference.csv",
    "./logs/results/rctd/results/0_resolution0.04_switch_reference.csv",
    "./logs/results/rctd/results/1_resolution0.04_switch_reference.csv",
]
rctd_results = []
for path in rctd_result_paths:
    result = pd.read_csv(path)
    result = result.set_index("Unnamed: 0")
    # normlize to 1
    result = result.div(result.sum(axis=1), axis=0)
    rctd_results.append(result)
rctd_dataset_names = dataset_names_orig[0:5] + dataset_names_orig[8:9] + dataset_names_orig[-8:-2]
assert(len(rctd_results) == len(rctd_dataset_names))
pprint(rctd_dataset_names)
rctd_names = ["RCTD" for i in rctd_results]


['slideSeqv2-kidney-2a',
 'slideSeqv2-kidney-4b',
 'seqFISH-heart-1',
 'seqFISH-heart-2',
 'seqFISH-heart-3',
 'Visium-lymph-node',
 'slideSeqv2-kidney-2a (sc. ref.)',
 'slideSeqv2-kidney-4b (sc. ref.)',
 'seqFISH-heart-1 (switched)',
 'seqFISH-heart-2 (switched)',
 'MERFISH-hypothalamus-0 (switched)',
 'MERFISH-hypothalamus-1 (switched)']


In [88]:
rctd_results[-2].sum(axis=1)

Unnamed: 0
0_24     1.0
10_1     1.0
10_10    1.0
10_11    1.0
10_12    1.0
        ... 
9_5      1.0
9_6      1.0
9_7      1.0
9_8      1.0
9_9      1.0
Length: 614, dtype: float64

In [78]:
ensemble_results[-2]

,Ambiguous,Astrocyte,Endothelial,Ependymal,Excitatory,Inhibitory,Microglia,OD Immature,OD Mature,Pericytes
0_24,0.027830,0.801557,0.023429,0.035927,0.018394,0.012080,0.027684,0.014821,0.016835,0.021443
10_1,0.207189,0.149607,0.008871,0.021699,0.101984,0.146729,0.235791,0.019147,0.101750,0.007233
10_10,0.176752,0.060129,0.036213,0.062456,0.256379,0.167512,0.030194,0.192072,0.007721,0.010573
10_11,0.230561,0.096419,0.290157,0.019828,0.107266,0.138709,0.007578,0.078822,0.008212,0.022448
10_12,0.068418,0.017513,0.025830,0.486516,0.185198,0.161678,0.007880,0.024473,0.012870,0.009624
...,...,...,...,...,...,...,...,...,...,...
9_5,0.117056,0.012264,0.121687,0.019711,0.571126,0.037500,0.012260,0.095141,0.006229,0.007025
9_6,0.114518,0.007397,0.254880,0.009765,0.500652,0.071456,0.013288,0.015973,0.006084,0.005986
9_7,0.199163,0.035689,0.073889,0.022863,0.362039,0.132103,0.115939,0.041369,0.006523,0.010422
9_8,0.248504,0.152909,0.102909,0.040753,0.136810,0.156350,0.016593,0.119264,0.008904,0.017005


### Load Cell2location results

### Load Linear baseline results

In [54]:
linear_runs = get_runs_for_tags_and_filters(baseline_tag="hard-exp", project="ssl-decon")
linear_results = [get_result_for_run(run) for run in linear_runs]
linear_dataset_names = [get_dataset_name_for_run(run, dataset_map, dataset_path_map) for run in linear_runs]
linear_names = ["Linear baseline"] * len(linear_runs)
print(linear_dataset_names)

Loaded 2 runs
['slideSeqv2-kidney-2a (sc. ref.)', 'slideSeqv2-kidney-4b (sc. ref.)']


### Load GraphST results

In [44]:
graph_st_result_paths = [
    "/data/GraphST/Data/slideseq_105_result.h5ad",
    "/data/GraphST/Data/slideseq_75_result.h5ad",
    "/data/GraphST/Data/seqfish_result.h5ad",
]
graphst_dataset_names = ["slideSeq-4b", "slideSeq-2a", "seqFISH"]
graph_st_names = len(dataset_names) * ["GraphST"]
graph_st_results = []
for path in graph_st_result_paths:
    result = sc.read_h5ad(path)
    graph_st_results.append(result.obs[result.obs.columns[2::]])
    print(len(result.obs.index))

1803
3244
1640


### Combine results

In [104]:
extra_results = [
    ensemble_results,
    # graph_st_results,
    rctd_results,
    linear_results,
]
extra_names = [
    dissect_names,
    # graph_st_names,
    rctd_names,
    linear_names,
]
extra_dataset_names = [
    dissect_dataset_names,
    # graphst_dataset_names,
    rctd_dataset_names,
    linear_dataset_names,
]
extra_results = [item for sublist in extra_results for item in sublist]
extra_names = [item for sublist in extra_names for item in sublist]
extra_dataset_names = [item for sublist in extra_dataset_names for item in sublist]

In [105]:
print(len(extra_results))

29


## Evaluate results

In [108]:
project = "multi-channel-gnn"
baseline_tag = "latestv2-switched"
# baseline_tag = "latestv2"
ablation_tag = "ablation-hard"
ablation_tag = None
dataset_filter = dataset_names_orig[0:5]# + dataset_names_orig[5:7]
dataset_filter = dataset_names_orig[0:0] + dataset_names_orig[8:8] + dataset_names_orig[-8:-6]
dataset_filter = dataset_names_orig[-8:-6]
print(dataset_filter)

['slideSeqv2-kidney-2a (sc. ref.)', 'slideSeqv2-kidney-4b (sc. ref.)']


In [107]:
name = None
import warnings
# warnings.simplefilter('error')
warnings.simplefilter("default")
id_values = {
    # "net/encoder_kwargs/mha_channel": [False, True],
    "model/beta": [0.0, 7.5, "v2", "v3"]
}
save_path = f"./figures/performance_comparison_no_slideseq.png"
(
    tabular_results,
    tabular_results_per_dataset,
    mean_results,
    std_results,
    all_cellwise_results,
    all_samplewise_results,
    methods_df,
) = evaluation_pipeline(
    dataset_map,
    dataset_path_map,
    id_values,
    name,
    extra_results=extra_results,
    extra_dataset_names=extra_dataset_names,
    extra_method_names=extra_names,
    baseline_tag=baseline_tag,
    extra_tag=ablation_tag,
    project=project,
    metrics=["Correlation", "CCC", "RMSE"],
    dataset_filter=dataset_filter,
    # save_path=save_path,
    # add_line_plot=True,
    use_bar_plot=False,
    return_all_dfs=True,
    show=False,
)
display(tabular_results)
display(tabular_results_per_dataset)

Loaded 8 runs


  0%|          | 0/8 [00:00<?, ?it/s]/data/DISSECT/src/utils/wandb.py:44: ResourceWarning: unclosed file <_io.TextIOWrapper name='./media/table/predictions-step-5000_519_e93918141f9f5b6b4873.table.json' mode='r' encoding='UTF-8'>
  path = result_file.download(replace=False, exist_ok=True).name
 12%|█▎        | 1/8 [00:00<00:01,  4.13it/s]/data/DISSECT/src/utils/wandb.py:44: ResourceWarning: unclosed file <_io.TextIOWrapper name='./media/table/predictions-step-5000_519_9877435aadbef7f203f7.table.json' mode='r' encoding='UTF-8'>
  path = result_file.download(replace=False, exist_ok=True).name
 25%|██▌       | 2/8 [00:00<00:01,  4.00it/s]/data/DISSECT/src/utils/wandb.py:44: ResourceWarning: unclosed file <_io.TextIOWrapper name='./media/table/predictions-step-5000_519_95131df355272b34e535.table.json' mode='r' encoding='UTF-8'>
  path = result_file.download(replace=False, exist_ok=True).name
 38%|███▊      | 3/8 [00:00<00:01,  4.24it/s]/data/DISSECT/src/utils/wandb.py:44: ResourceWarning: 

Number of methods: 8
Adding 29 extra results to evaluation...


 76%|███████▋  | 13/17 [00:06<00:02,  1.70it/s]/root/miniconda3/envs/dissect-spatial/lib/python3.8/site-packages/numpy/lib/function_base.py:2829: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/root/miniconda3/envs/dissect-spatial/lib/python3.8/site-packages/numpy/lib/function_base.py:2830: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
100%|██████████| 17/17 [00:09<00:00,  1.71it/s]
/data/DISSECT/src/utils/plotting.py:19: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  mpl.style.use("seaborn-paper")


Number of special names: 0


/data/DISSECT/src/utils/plotting.py:106: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  mpl.style.use("seaborn-paper")
/tmp/ipykernel_812567/1439749491.py:18: ResourceWarning: unclosed <ssl.SSLSocket fd=69, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.17.0.2', 44746), raddr=('172.22.1.7', 8888)>
  ) = evaluation_pipeline(


,Correlation,RMSE,CCC,Correlation (samplewise),RMSE (samplewise),CCC (samplewise),JSD
Method,,,,,,,
Baseline,0.724829,0.077985,0.631783,0.853923,0.084360,0.759427,0.147803
DISSECT (ensemble),0.708895,0.087838,0.562114,0.822229,0.099478,0.658279,0.193127
RCTD,0.714917,0.071127,0.682507,0.848425,0.072111,0.827040,0.107338


Correlation      RMSE  \
Method             dataset                                                    
Baseline           MERFISH-hypothalamus-0 (switched)     0.835598  0.067497   
                   MERFISH-hypothalamus-1 (switched)     0.825118  0.067730   
                   seqFISH-heart-1 (switched)            0.669407  0.096945   
                   seqFISH-heart-2 (switched)            0.569191  0.079768   
DISSECT (ensemble) MERFISH-hypothalamus-0 (switched)     0.824750  0.072772   
                   MERFISH-hypothalamus-1 (switched)     0.814931  0.073464   
                   seqFISH-heart-1 (switched)            0.646601  0.112411   
                   seqFISH-heart-2 (switched)            0.549299  0.092703   
RCTD               MERFISH-hypothalamus-0 (switched)     0.800683  0.080128   
                   MERFISH-hypothalamus-1 (switched)     0.797659  0.077453   
                   seqFISH-heart-1 (switched)            0.658671  0.076811   
                   seqFISH-heart-2 (switched)            0.602657  0.050118   

                                                           CCC  \
Method             dataset                                       
Baseline           MERFISH-hypothalamus-0 (switched)  0.794353   
                   MERFISH-hypothalamus-1 (switched)  0.788025   
                   seqFISH-heart-1 (switched)         0.498028   
                   seqFISH-heart-2 (switched)         0.446724   
DISSECT (ensemble) MERFISH-hypothalamus-0 (switched)  0.767784   
                   MERFISH-hypothalamus-1 (switched)  0.765516   
                   seqFISH-heart-1 (switched)         0.376571   
                   seqFISH-heart-2 (switched)         0.338587   
RCTD               MERFISH-hypothalamus-0 (switched)  0.775025   
                   MERFISH-hypothalamus-1 (switched)  0.780001   
                   seqFISH-heart-1 (switched)         0.611778   
                   seqFISH-heart-2 (switched)         0.563222   

                                                      Correlation (samplewise)  \
Method             dataset                                                       
Baseline           MERFISH-hypothalamus-0 (switched)                  0.848985   
                   MERFISH-hypothalamus-1 (switched)                  0.857463   
                   seqFISH-heart-1 (switched)                         0.813022   
                   seqFISH-heart-2 (switched)                         0.896223   
DISSECT (ensemble) MERFISH-hypothalamus-0 (switched)                  0.816064   
                   MERFISH-hypothalamus-1 (switched)                  0.824097   
                   seqFISH-heart-1 (switched)                         0.777807   
                   seqFISH-heart-2 (switched)                         0.870948   
RCTD               MERFISH-hypothalamus-0 (switched)                  0.797497   
                   MERFISH-hypothalamus-1 (switched)                  0.824022   
                   seqFISH-heart-1 (switched)                         0.843781   
                   seqFISH-heart-2 (switched)                         0.928402   

                                                      RMSE (samplewise)  \
Method             dataset                                                
Baseline           MERFISH-hypothalamus-0 (switched)           0.071168   
                   MERFISH-hypothalamus-1 (switched)           0.072487   
                   seqFISH-heart-1 (switched)                  0.097740   
                   seqFISH-heart-2 (switched)                  0.096045   
DISSECT (ensemble) MERFISH-hypothalamus-0 (switched)           0.079866   
                   MERFISH-hypothalamus-1 (switched)           0.082669   
                   seqFISH-heart-1 (switched)                  0.120730   
                   seqFISH-heart-2 (switched)                  0.114646   
RCTD               MERFISH-hypothalamus-0 (switched)           0.087883   
                   MERFISH-hypothalamus-1 (switc

### Further visualizations

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 5), sharey="row", sharex="col")
axs = np.ravel(axs)
metric = "Correlation"
relevant_methods = ["DISSECT (ensemble)", "gnn-3"]
dataset = "slideSeq-4b"
methods_mask = all_cellwise_results["Method"].isin(relevant_methods)
data_mask = all_cellwise_results["dataset"] == dataset
# sns.barplot(all_cellwise_results.loc[methods_mask, :], x="Celltype", y=metric, hue="Method", ax=axs[0])
axs[0].set_title("Performance per celltype for selected methods")
sns.barplot(all_cellwise_results.loc[methods_mask & data_mask, :], x="Celltype", y=metric, hue="Method", ax=axs[1])
for ax in axs[-2:]:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=30, ha="right")
for ax in axs:
    ax.grid(True)
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 5), sharey="row", sharex="col")
axs = np.ravel(axs)
metric = "Correlation"
relevant_methods = ["DISSECT (ensemble)", "gnn-3"]
methods_mask = all_cellwise_results["Method"].isin(relevant_methods)
sns.barplot(all_cellwise_results.loc[methods_mask, :], x="Celltype", y=metric, hue="Method", ax=axs[0])
axs[0].set_title("Performance per celltype for selected methods")
sns.barplot(all_cellwise_results, x="Method", y=metric, hue="dataset", ax=axs[1])
for ax in axs[-2:]:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=30, ha="right")
for ax in axs:
    ax.grid(True)
plt.show()

### Single dataset analysis

In [ ]:
cellwise_results = compare_methods_new(
    all_results, groundtruth, methods=all_method_names, samplewise=False
)
samplewise_results = compare_methods_new(
    all_results, groundtruth, methods=all_method_names, samplewise=True
)

In [ ]:
cellwise_grouped = cellwise_results.groupby(["Method", "Fold"]).mean(numeric_only=True)
samplewise_grouped = samplewise_results.groupby(["Method", "Fold"]).mean(numeric_only=True)
pd.concat([cellwise_grouped, samplewise_grouped], axis=1)

In [ ]:
# plot performance per celltype
fig, axs = plt.subplots(2, 2, figsize=(16, 10), sharey="row", sharex="col")
axs = np.ravel(axs)
sns.barplot(cellwise_results, x="Celltype", y="Correlation", hue="Method", ax=axs[0])
mean_cellwise_results = cellwise_results.groupby(
    ["Method", "Fold"], as_index=False
).mean(numeric_only=True)
sns.barplot(mean_cellwise_results, x="Method", y="Correlation", ax=axs[1])

sns.barplot(cellwise_results, x="Celltype", y="RMSE", hue="Method", ax=axs[2])
mean_cellwise_results = cellwise_results.groupby(
    ["Method", "Fold"], as_index=False
).mean(numeric_only=True)
sns.barplot(mean_cellwise_results, x="Method", y="RMSE", ax=axs[3])
# rotate xticks
for ax in axs[-2:]:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=30, ha="right")
for ax in axs:
    ax.grid(True)

plt.savefig(
    f"./figures/evaluation_{experiment_name}_cellwise.png", dpi=200, bbox_inches="tight"
)
plt.show()

In [ ]:
# samplewise results
# plot performance per celltype
metrics = ["Correlation", "RMSE"]
for metric in metrics:
    fig, axs = plt.subplots(1, 2, figsize=(16, 5), sharey="row", sharex="col")
    sns.boxplot(
        samplewise_results,
        x="Method",
        y=metric + " (samplewise)",
        hue="Fold",
        ax=axs[0],
    )
    mean_samplewise_results = samplewise_results.groupby(
        ["Method", "Fold"], as_index=False
    ).mean()
    sns.barplot(
        mean_samplewise_results, x="Method", y=metric + " (samplewise)", ax=axs[1]
    )
    # plt.savefig(
    #     f"./figures/evaluation_{experiment_name}_cellwise.png", dpi=200, bbox_inches="tight"
    # )
    plt.show()

## Qualitative analysis

In [ ]:
# load data
st_path = "./data/spatial/V1_Mouse_Brain_Sagittal_Anterior.h5ad"
st_data = sc.read_h5ad(st_path)

# run_name = "super-armadillo-244"
# run_name = "captain-maquis-285"
run_name = "defiant-frontier-290"
run_name = "xindi-q-292"
run_name = "robust-sun-298"
result = get_result_for_run_name(run_name, project="dissect-spatial")

st_data.obs = pd.concat([st_data.obs, result], axis=1)

In [ ]:
# for col in result.columns:
#     fig, axs = plt.subplots(1, 1, figsize=(10, 5))
# sc.pl.spatial(st_data, color=result.columns, show=True, save=f"mouse_brain_new_v2.png", vmin=0.0, vmax=1.0)
fig = sc.pl.spatial(st_data, color=result.columns, show=False, save=False, vmin=0.0, vmax=1.0)
plt.savefig("./figures/mouse.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# load data
st_path = "./data/spatial/V1_Mouse_Brain_Sagittal_Anterior.h5ad"
st_data = sc.read_h5ad(st_path)

dissect_result = load_dissect_results("experiments/experiment_mouse_st")[1]
st_data.obs = pd.concat([st_data.obs, dissect_result], axis=1)
fig = sc.pl.spatial(st_data, color=result.columns, show=False, save=False, vmin=0.0, vmax=1.0)
plt.savefig("./figures/mouse_brain_dissect_ensemble.png", dpi=300, bbox_inches="tight")
plt.show()

## Old analysis

### Predicted max celltype per spot

In [ ]:
for k, df in enumerate(results_per_seed):
    max_celltypes = df.idxmax(axis=1)
    st_data.obs[f"celltype-{k}"] = max_celltypes
    sq.pl.spatial_scatter(st_data, color=f"celltype-{k}")

### Celltype distribution across tissue

In [ ]:
st_data.obs[celltypes] = new_result[celltypes]
sq.pl.spatial_scatter(st_data, color="L5 IT")
plt.savefig(
    f"{base_path}/figures/single_celltype_{experiment_name}.png",
    dpi=200,
    bbox_inches="tight",
)

In [ ]:
st_data.obs[celltypes] = ensemble_result[celltypes]
sq.pl.spatial_scatter(st_data, color=celltypes)

### Comparison between dissect runs for different seeds

In [ ]:
all_corrs = []
for df_1 in results_per_seed:
    for df_2 in results_per_seed:
        all_corrs.append(calc_mean_corr_df(df_1, df_2, verbose=1)[0])
    # calc correlation
all_corrs = np.reshape(all_corrs, (5, 5))

In [ ]:
labels = [f"Seed {i}" for i in range(5)]

In [ ]:
def heatmap(
    data, row_labels, col_labels, ax=None, cbar_kw=None, cbarlabel="", **kwargs
):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (M, N).
    row_labels
        A list or array of length M with the labels for the rows.
    col_labels
        A list or array of length N with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if ax is None:
        ax = plt.gca()

    if cbar_kw is None:
        cbar_kw = {}

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # Show all ticks and label them with the respective list entries.
    ax.set_xticks(np.arange(data.shape[1]), labels=col_labels)
    ax.set_yticks(np.arange(data.shape[0]), labels=row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False, labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right", rotation_mode="anchor")

    # Turn spines off and create white grid.
    ax.spines[:].set_visible(False)

    ax.set_xticks(np.arange(data.shape[1] + 1) - 0.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0] + 1) - 0.5, minor=True)
    ax.grid(which="minor", color="w", linestyle="-", linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(
    im,
    data=None,
    valfmt="{x:.2f}",
    textcolors=("black", "white"),
    threshold=None,
    **textkw
):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A pair of colors.  The first is used for values below a threshold,
        the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max()) / 2.0

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center", verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

In [ ]:
fig, ax = plt.subplots()

im, cbar = heatmap(
    all_corrs, labels, labels, ax=ax, cmap="Blues", cbarlabel="Pearson correlation"
)
texts = annotate_heatmap(im, valfmt="{x:.2f}")
ax.set_title("Correlation between seeded runs in original DISSECT")

fig.tight_layout()
plt.savefig(
    "../figures/corr_heatmap_dissect_original.png", dpi=200, bbox_inches="tight"
)
plt.show()

### Comparison with new implementation

In [ ]:
new_corrs = []
for df in results_per_seed:
    new_corrs.append(calc_mean_corr_df(df, new_result)[0])
new_corrs = np.reshape(new_corrs, (5, 1))

In [ ]:
fig, ax = plt.subplots()
ax.boxplot(new_corrs)
ax.set_xticklabels(" ")
ax.set_title("Mean correlation between seeded runs and new DISSECT")
fig.tight_layout()
plt.savefig("../figures/corr_dissect_new_vs_old.png", dpi=200, bbox_inches="tight")
plt.show()

In [ ]:
print(np.mean(new_corrs))
print(np.std(new_corrs))
print(np.max(new_corrs))
print(np.min(new_corrs))

## Compare per cell type

In [ ]:
(
    mean_corr,
    corrs,
    mean_corr_sample,
    corrs_sample,
    mean_rmse,
    rmses,
    mean_rmse_sample,
    rmses_sample,
) = calc_metrics_df(ensemble_result, y_real, verbose=1, exclude_cols=None)

In [ ]:
# plot corrs per cell type
fig, ax = plt.subplots()
cell_types = y_real.columns
ax.bar(cell_types, corrs)

In [ ]:
for col in ensemble_result.columns:
    st_data.obs[f"{col} pred"] = ensemble_result[col]

In [ ]:
for col in ensemble_result.columns:
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    sc.pl.spatial(st_data, color=col, ax=axs[0], show=False, cmap="Reds")
    sc.pl.spatial(st_data, color=f"{col} pred", ax=axs[1], cmap="Reds")

In [ ]:
for col in ensemble_result.columns:
    st_data.obs[f"{col} pred"] = ensemble_result[col]